### Interactive Corporate Credit Risk Analysis

This notebook provides an interactive tool for corporate credit risk analysis. It uses a simplified S&P framework, key financial metrics, and DCF/EV calculations to generate a credit rating report and simplified financial outputs. Users can input company-specific data to receive a personalized analysis.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### User Input Section

Below, you can input the company's financial data and assumptions for the analysis.

In [ ]:
# --- Input Widgets ---
company_name_input = widgets.Text(description='Company Name')
ticker_symbol_input = widgets.Text(description='Ticker Symbol')

revenue_input = widgets.FloatText(description='Revenue')
ebitda_input = widgets.FloatText(description='EBITDA')
cash_input = widgets.FloatText(description='Cash')
total_debt_input = widgets.FloatText(description='Total Debt')
capex_input = widgets.FloatText(description='Capex')
interest_expense_input = widgets.FloatText(description='Interest Expense')
market_cap_input = widgets.FloatText(description='Market Cap')  # For EV
growth_rate_input = widgets.FloatText(description='Growth Rate (%)')  # For DCF
discount_rate_input = widgets.FloatText(description='Discount Rate (%)')  # For DCF
terminal_growth_input = widgets.FloatText(description='Terminal Growth (%)')  # For DCF

# --- Helper Function to Display Inputs ---
def display_inputs():
    display(company_name_input, ticker_symbol_input, revenue_input, ebitda_input, cash_input,
            total_debt_input, capex_input, interest_expense_input, market_cap_input, growth_rate_input,
            discount_rate_input, terminal_growth_input)

### Input Validation and Error Handling

The following code validates the user inputs to ensure they are positive and reasonable.

In [ ]:
# --- Input Validation Function ---
def validate_inputs(data):
    """
    Validates the user input data to ensure they are positive and reasonable.
    """
    for field, value in data.items():
        if value <= 0:
            raise ValueError(f"{field} must be a positive value.")
    return True

### Calculations

This section contains the calculations for financial ratios, credit rating predictions, DCF, and EV analysis.

In [ ]:
# --- Calculation Functions ---
def calculate_ratios(data):
    """
    Calculates key financial ratios.
    """
    debt_to_ebitda = data['Total Debt'] / data['EBITDA'] if data['EBITDA'] != 0 else 0
    interest_coverage = data['EBITDA'] / data['Interest Expense'] if data['Interest Expense'] != 0 else 0
    debt_to_equity = data['Total Debt'] / data['Cash'] if data['Cash'] != 0 else 0
    ebitda_margin = data['EBITDA'] / data['Revenue'] if data['Revenue'] != 0 else 0

    return {
        'Debt-to-EBITDA': debt_to_ebitda,
        'Interest Coverage': interest_coverage,
        'Debt-to-Equity': debt_to_equity,
        'EBITDA Margin': ebitda_margin
    }

def predict_rating(ratios):
    """
    Predicts the credit rating based on calculated ratios.
    """
    debt_to_ebitda = ratios['Debt-to-EBITDA']
    interest_coverage = ratios['Interest Coverage']
    ebitda_margin = ratios['EBITDA Margin']

    if debt_to_ebitda < 2.5 and interest_coverage > 5 and ebitda_margin > 0.2:
        rating = 'AA'
    elif 2.5 <= debt_to_ebitda <= 3.5 and interest_coverage > 3 and ebitda_margin > 0.15:
        rating = 'A'
    elif 3.5 < debt_to_ebitda <= 4.5 and interest_coverage > 2 and ebitda_margin > 0.1:
        rating = 'BBB'
    else:
        rating = 'BB'

    return rating

def calculate_dcf(data):
    """
    Performs a simplified Discounted Cash Flow (DCF) calculation.
    """
    fcf = data['EBITDA'] - data['Capex']
    pv_fcf = 0
    for year in range(1, 6):
        fcf *= (1 + data['Growth Rate'])
        pv_fcf += fcf / (1 + data['Discount Rate'])**year
    terminal_value = fcf / data['Discount Rate']
    pv_terminal_value = terminal_value / (1 + data['Discount Rate'])**5
    enterprise_value = pv_fcf + pv_terminal_value
    return {
        'Present Value of FCF': pv_fcf,
        'Terminal Value': pv_terminal_value,
        'Enterprise Value': enterprise_value
    }

def calculate_ev(data):
    """
    Performs a simplified Enterprise Value (EV) calculation.
    """
    return data['Market Cap'] + data['Total Debt'] - data['Cash']

### Visualizations

The following plot displays the results of the DCF and EV calculations.

In [ ]:
# --- Plotting Function ---
def plot_results(dcf_output, ev_output):
    """
    Plots the simplified DCF and EV results.
    """
    labels = ['Present Value of FCF', 'Terminal Value', 'Enterprise Value']
    values = [dcf_output['Present Value of FCF'], dcf_output['Terminal Value'], dcf_output['Enterprise Value']]
    plt.bar(labels, values)
    plt.title('Simplified DCF Results')
    plt.show()

### Dynamic Execution

Click the button below to calculate the credit rating and financial outputs.

In [ ]:
# --- Button Click Event ---
def on_button_click(b):
    try:
        # Retrieve data from user input
        user_data = {
            'Company Name': company_name_input.value,
            'Ticker Symbol': ticker_symbol_input.value,
            'Revenue': revenue_input.value,
            'EBITDA': ebitda_input.value,
            'Cash': cash_input.value,
            'Total Debt': total_debt_input.value,
            'Capex': capex_input.value,
            'Interest Expense': interest_expense_input.value,
            'Market Cap': market_cap_input.value,
            'Growth Rate': growth_rate_input.value / 100,  # Convert to decimal
            'Discount Rate': discount_rate_input.value / 100,  # Convert to decimal
            'Terminal Growth': terminal_growth_input.value / 100  # Convert to decimal
        }
        # Validate Inputs
        validate_inputs(user_data)
        # Calculate financial ratios
        ratios = calculate_ratios(user_data)
        # Predict Credit Rating
        rating = predict_rating(ratios)
        # Perform DCF and EV calculations
        dcf_output = calculate_dcf(user_data)
        ev_output = calculate_ev(user_data)
        # Display results
        display(f"Predicted Credit Rating: {rating}")
        display(f"Debt-to-EBITDA: {ratios['Debt-to-EBITDA']:.2f}")
        display(f"Interest Coverage: {ratios['Interest Coverage']:.2f}")
        display(f"Debt-to-Equity: {ratios['Debt-to-Equity']:.2f}")
        display(f"EBITDA Margin: {ratios['EBITDA Margin']:.2f}")
        display(f"Present Value of Free Cash Flows: {dcf_output['Present Value of FCF']:.2f}")
        display(f"Terminal Value: {dcf_output['Terminal Value']:.2f}")
        display(f"Enterprise Value: {dcf_output['Enterprise Value']:.2f}")
        display(f"Enterprise Value (EV): {ev_output:.2f}")
        # Plot Results
        plot_results(dcf_output, ev_output)
    except ValueError as e:
        display(f"Error: {e}")
        display_inputs()

# --- Add a Button to Trigger Calculations ---
button = widgets.Button(description='Calculate Credit Rating')
button.on_click(on_button_click)
display(button)